In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols 
from rasterstats import zonal_stats
import rasterio
import contextily as cx
from rasterio.plot import show
#import osmnx as ox
from affine import Affine
import rioxarray as rx
from multiprocessing import Pool, cpu_count
from shapely.ops import unary_union
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from pathlib import Path

In [2]:

#read in municipalities, select the one of interest
#municipalities
munis_fp = "K:\\DataServices\\Datasets\\Boundaries\\Spatial\\mapc_towns_poly.shp"
munis = gpd.read_file(munis_fp)

#land use geodatabase location
lclu_gdb = "K:\\DataServices\\Datasets\\Zoning and Land Use\\Spatial\\lclu_gdb\\MA_LCLU2016.gdb"

In [3]:
muni = 'Natick'

print('Starting on municipality: ', muni)

#make a gdf for muni feature class
muni_shp = munis.loc[munis['municipal'] == muni]
#read lclu but only for the muni (this makes it small enough to process)
lclu = gpd.read_file(lclu_gdb, layer="LANDCOVER_LANDUSE_POLY", mask=muni_shp)
    
#reproject all to mass mainland
mass_mainland_crs = "EPSG:26986"
lclu = lclu.to_crs(mass_mainland_crs)
muni_shp = muni_shp.to_crs(mass_mainland_crs)

Starting on municipality:  Natick


In [5]:
lclu.head()

#follow this guidance: https://www.mass.gov/doc/2016-massachusetts-small-ms4-permit-pollutant-loading-export-rates/download


print(lclu.COVERNAME.unique())

['Deciduous Forest' 'Evergreen Forest' 'Developed Open Space'
 'Palustrine Scrub/Shrub Wetland' 'Impervious'
 'Palustrine Forested Wetland' 'Palustrine Emergent Wetland' 'Grassland'
 'Bare Land' 'Scrub/Shrub' 'Water' 'Palustrine Aquatic Bed' 'Pasture/Hay'
 'Cultivated']


In [11]:
print(lclu.COVERNAME.unique())
print(lclu.USEGENNAME.unique())

['Deciduous Forest' 'Evergreen Forest' 'Developed Open Space'
 'Palustrine Scrub/Shrub Wetland' 'Impervious'
 'Palustrine Forested Wetland' 'Palustrine Emergent Wetland' 'Grassland'
 'Bare Land' 'Scrub/Shrub' 'Water' 'Palustrine Aquatic Bed' 'Pasture/Hay'
 'Cultivated']
['Residential - single family' 'Tax exempt' 'Right-of-way' 'Unknown'
 'Open land' 'Water' 'Residential - multi-family' 'Industrial'
 'Commercial' 'Recreation' 'Agriculture' 'Residential - other' 'Forest'
 'Mixed use, other' 'Mixed use, primarily residential']


In [22]:
#calculate area in acreage
lclu['acres'] = lclu.area / 43560


#distinguish between impervious and pervious
impervious = lclu.loc[lclu['COVERNAME'] == 'Impervious']
pervious = lclu.loc[lclu['COVERNAME'] != 'Impervious']

In [30]:
category_1_vals = ['Mixed use, primarily residential', 'Residential - multi-family']
category_2_vals = ['Commercial', 'Industrial', 'Tax exempt', 'Mixed use, other', 'Mixed use, primarily commercial'] 
category_3_vals = ['Right-of-way']
category_4_vals = ['Forest']
category_5_vals = ['Agriculture']
category_6_vals = ['Unknown', 'Open land', 'Recreation']
category_7_vals = ['Residential - other', 'Residential - single family']

cover_1_vals = ['Deciduous Forest', 'Evergreen Forest', 'Scrub/Shrub']
cover_2_vals = ['Developed Open Space', 'Grassland', 'Palustrine Forested Wetland', 
                'Palustrine Scrub/Shrub Wetland', 'Palustrine Emergent Wetland',
                'Estuarine Forested Wetland', 'Estuarine Scrub/Shrub Wetland', 'Estuarine Emergent Wetland'
                ]
cover_3_vals = ['Cultivated', 'Pasture/Hay']

PLER_rule = [
    (lclu['COVERNAME'] == 'Impervious') & lclu['USEGENNAME'].isin(category_1_vals),
    (lclu['COVERNAME'] == 'Impervious') & lclu['USEGENNAME'].isin(category_2_vals),
    (lclu['COVERNAME'] == 'Impervious') & lclu['USEGENNAME'].isin(category_3_vals),
    (lclu['COVERNAME'] == 'Impervious') & lclu['USEGENNAME'].isin(category_4_vals),
    (lclu['COVERNAME'] == 'Impervious') & lclu['USEGENNAME'].isin(category_5_vals),
    (lclu['COVERNAME'] == 'Impervious') & lclu['USEGENNAME'].isin(category_6_vals),
    (lclu['COVERNAME'] == 'Impervious') & lclu['USEGENNAME'].isin(category_7_vals),
    (lclu['COVERNAME'] != 'Impervious') &  lclu['COVERNAME'].isin(cover_1_vals),
    (lclu['COVERNAME'] != 'Impervious') & (lclu['USEGENNAME'] == 'Forest') & lclu['COVERNAME'].isin(cover_2_vals),
    (lclu['COVERNAME'] != 'Impervious') &  lclu['COVERNAME'].isin(cover_3_vals),
    (lclu['COVERNAME'] != 'Impervious') & (lclu['COVERNAME'] == 'Developed Open Space') & (lclu['USEGENNAME']== 'Agriculture'),
    (lclu['COVERNAME'] != 'Impervious') & (lclu['COVERNAME'] == 'Grassland') & (lclu['USEGENNAME']== 'Agriculture'),
    (lclu['COVERNAME'] != 'Impervious') & (lclu['COVERNAME'] == 'Pasture/Hay') & (lclu['USEGENNAME']== 'Forest')
]

choices = [2.32, 1.78, 1.95, 1.52, 1.52, 1.52, 1.96, 0.13, 0.13, 0.45, 0.45, 0.45, 0.45]

lclu['PLER'] = np.select(PLER_rule, choices, default=np.nan)
lclu[['COVERNAME', 'USEGENNAME', 'PLER', 'acres']]



,COVERNAME,USEGENNAME,PLER,acres
0,Deciduous Forest,Residential - single family,0.13,7.035807e-03
1,Deciduous Forest,Tax exempt,0.13,1.304229e-03
2,Evergreen Forest,Right-of-way,0.13,2.035803e-03
3,Evergreen Forest,Tax exempt,0.13,1.881605e-03
4,Evergreen Forest,Unknown,0.13,8.930212e-05
...,...,...,...,...
92639,Developed Open Space,Residential - single family,NaN,1.619255e-02
92640,Palustrine Forested Wetland,Residential - single family,NaN,1.361576e-03
92641,Impervious,Residential - single family,1.96,2.352911e-03
92642,Deciduous Forest,Right-of-way,0.13,1.937693e-03


In [ ]:
cover_4_vals = ['Developed Open Space', 'Grassland', 'Palustrine Forested Wetland', 'Palustrine Scrub/Shrub Wetland',
                ]

In [ ]:
['Deciduous Forest' 'Evergreen Forest' 'Developed Open Space'
 'Palustrine Scrub/Shrub Wetland' 'Impervious'
 'Palustrine Forested Wetland' 'Palustrine Emergent Wetland' 'Grassland'
 'Bare Land' 'Scrub/Shrub' 'Water' 'Palustrine Aquatic Bed' 'Pasture/Hay'
 'Cultivated']
['Residential - single family' 'Tax exempt' 'Right-of-way' 'Unknown'
 'Open land' 'Water' 'Residential - multi-family' 'Industrial'
 'Commercial' 'Recreation' 'Agriculture' 'Residential - other' 'Forest'
 'Mixed use, other' 'Mixed use, primarily residential']